# ASR Architectures

## Seq2Seq architecture

![Image Title](images/whisper.png)
*Example of a Seq2Seq architecture — Whisper model*

On the left is the transformer encoder. It takes a log-mel spectrogram as input and encodes that spectrogram to form a sequence of encoder hidden states that extract essential features from the spoken speech. This hidden states tensor represents the input sequence as a whole and effectively encodes the "meaning" of the input speech.

The encoder's output is then passed into the transformer decoder, shown on the right, using a cross-attention mechanism. It is like self-attention, but it attends to the encoder's output. From this point on, the encoder is no longer used.

The decoder predicts a sequence of text tokens in an autoregressive manner, a single token at a time, starting from an initial sequence with a "start" token in it (`SOT` in the case of Whisper). At each following timestep, the previous output sequence is fed back into the decoder as the new input sequence. In this manner, the decoder emits one new token at a time, steadily growing the output sequence until it predicts the "end" token or a maximum number of timesteps is reached.

While the architecture of the decoder is primarily identical to the encoder, there are two significant differences:
1. The decoder has a cross-attention mechanism that allows it to look at the encoder's representation of the input sequence.
2. The decoder's attention is causal — the decoder isn't allowed to look into the future.

A typical loss function for a seq2seq ASR model is the cross-entropy loss, as the final layer of the model predicts a probability distribution over the possible tokens.

## CTC architecture

**CTC** or Connectionist Temporal Classification is a technique that is used with encoder-only transformer models for automatic speech recognition. Examples of such models are Wav2Vec2, HuBERT and M-CTC-T.

### Wav2Vec2

![Wav2Vec description](images/wav2vec2.png)

**Feature encoder**: This is the encoder part of the model. It takes the raw audio data as input and outputs feature vectors. Input size is limited to 400 samples, which is 20ms for a 16kHz sample rate. The raw audio is first standardized to have zero mean and unit variance. Then, it is passed to the 1D convolutional neural network (temporal convolution), followed by layer normalization and GELU activation function. There could be seven such convolution blocks with constant channel size (512), decreasing kernel width (10, 3x4, 2x2), and stride (5, 2x6). The output is a list of feature vectors, each with 512 dimensions.

**Transformers**: The output of the feature encoder is passed on to a transformer layer. One differentiator is the use of relative positional embedding by using convolution layers rather than fixed positional encoding, as done in the original Transformers paper. The block size differs, as 12 transformer blocks with a model dimension of 768 are used in the BASE model but 24 blocks with a dimension of 1024 in the LARGE version.

**Quantization module**: We need to work with discrete outputs for self-supervised learning. For this, there is a quantization module that converts the continuous vector output to discrete representations, and on top of it, it automatically learns the discrete speech units. It is done by maintaining multiple codebooks/groups (320 in size), and the units sampled from each codebook are later concatenated (320x320=102,400 possible speech units). The sampling is done using Gumbel-Softmax, which is like argmax but differentiable.

> **Wav2Vec2 as pre-training + CTC as fine-tuning = ASR :)**

An **encoder-only transformer** is the simplest kind of transformer because it uses only the encoder portion of the model. The encoder reads the input sequence (the audio waveform) and maps it into a sequence of hidden states, also known as the output embeddings.

With a CTC model, we apply an additional linear mapping to the sequence of hidden states to get class label predictions. The class labels are the characters of the alphabet (a, b, c, …). This way, we can predict any word in the target language with a small classification head, as the vocabulary must consist of 26 characters plus a few special tokens.

![Image Title](images/ctc.png)

Let's suppose our input is a one-second audio file. In Wav2Vec2, the model first downsamples the audio input using the CNN feature encoder to a shorter sequence of hidden states. One hidden state vector corresponds to 20 milliseconds of audio. We forward a sequence of 50 hidden states to the transformer encoder for one second of audio. (The audio segments extracted from the input sequence partially overlap, so even though one hidden state vector is emitted every 20 ms, each hidden state actually represents 25 ms of audio.)

The transformer encoder predicts one feature representation for each hidden state, meaning we receive 50 outputs from the transformer. Each of these outputs has a dimensionality of 768. Therefore, the transformer encoder's output sequence in this example has a shape (768, 50). As each of these predictions covers 25 ms of time, which is shorter than the duration of a phoneme, it makes sense to predict individual phonemes or characters but not entire words. CTC works best with a small vocabulary, so we'll predict characters.

![Image Title](images/ctc1.png)

To make text predictions, we map each of the 768-dimensional encoder outputs to our character labels using a linear layer (the "CTC head"). The model then predicts a (50, 32) tensor containing the logits, where 32 is the number of tokens in the vocabulary. Since we make one prediction for each of the features in the sequence, we end up with a total of 50 character predictions for each second of audio.

### A decoding algorithm and a loss function

The intuition of CTC is to output a single character for every frame of the input so that the output is the same length as the input and then to apply a collapsing function that combines sequences of identical letters, resulting in a shorter sequence.

Let's imagine inference on someone saying the word dinner, and suppose we had a function that chooses the most probable letter for each input spectral frame representation $x_i$. We'll call the sequence of letters corresponding to each input frame an alignment because it tells us where each letter aligns in the acoustic signal. **Figure 1** shows one such alignment and what happens if we use a collapsing function that just removes consecutive duplicate letters.

![Image Title](images/basic_decoding.png)

***Figure 1**: A naive algorithm for collapsing an alignment between input and letters.*

That doesn't work; our naive algorithm has transcribed the speech as a `diner`, not `dinner`! Collapsing doesn't handle double letters. Another problem with our naive function is that it doesn't tell us what symbol to align with silence in the input. We don't want to be transcribing silence as random letters!

#### The key decoding problem
1. We have to decode `RRROODD`.
2. Is it a `ROD` or `ROOD`?

The CTC algorithm solves both problems by adding a special symbol for a blank to the transcription alphabet, which we'll represent `⌴`. The blank can be used in the alignment whenever we don't want to transcribe a letter. It also can be used between letters; since our collapsing function collapses only consecutive duplicate letters, it won't collapse across `⌴`. More formally, let's define the mapping $B : a → y$ between an alignment $a$ and an output $y$, which collapses all repeated letters and then removes all blanks. **Figure 2** sketches this collapsing function $B$.

The symbol set that the network can recognise must include `⌴`.

![Image Title](images/ctc_collapsing_function.png)

***Figure 2**: The CTC collapsing function $B$, showing the space blank character `⌴`; repeated (consecutive) characters in an alignment $A$ are removed to form the output $Y$.*

The CTC collapsing function is many-to-one; many different alignments map to the exact output string. For example, the alignment shown in **Figure 2** is not the only one resulting in the string dinner. **Figure 2** shows some other alignments that would produce the same output. We'll use the inverse of our B function, called $B^{-1}$, and represent that set as $B^{-1}(Y)$.

![Three other legitimate alignments producing the transcript `dinner`](images/other_alignments.png)

***Figure 3**: Three other legitimate alignments producing the transcript `dinner`.*

### CTC Inference

Before we see how to compute $P_{CTC}(Y|X)$, let's first see how CTC assigns a probability to one particular alignment $\hat{A} = { \hat{a}_1, \ldots, \hat{a}_n }$. CTC makes a strong conditional independence assumption: it assumes that, given the input $X$, the CTC model output $a_t$ at time $t$ is independent of the output labels at any other time $a_i$. Thus:
$$P_{\text{ctc}}(A|X) = \prod_{t=1}^{T} p(a_t | X)$$

Thus to find the best alignment $\hat{A} = { \hat{a}_1, \ldots, \hat{a}_n }$, we can greedily choose the character with the max probability at each time step t: 
$$a_t = \text{argmax}_{c \in C} p_t(c | X)$$

We then pass the resulting sequence $A$ to the CTC collapsing function $B$ to get the output sequence $Y$.

We implemented the Best path decoding. It is the simplest method to decode the output matrix:
1. Concatenate most probable characters per timestep which yields the best path.
2. Then, undo the encoding by removing duplicate characters and all blanks. This gives us the recognized text.

In **Figure 4**, you can see an example of the Best path decoding.

![Image Title](images/most_likely_characters.png)

***Figure 4**: Concatenate most probable characters per timestep to get best path.*

#### Why best path decoding can fail

![Image Title](images/all_paths_to_a.png)

***Figure 5**: All paths corresponding to text "a".*

Best path decoding is both fast and simple, which are of course nice properties. But it may fail in certain situations like the one shown in **Fig 2**. In **Fig. 4** all paths corresponding to the text “a” are shown: `aa`, `a⌴` and `⌴a`. The probability of the text `a` is the sum over all probabilities of these mentioned paths: 0.2·0.4+0.2·0.6+0.8·0.4=0.52. So, `a` is more probable than `⌴` (0.52>0.48). We need a better algorithm than best path decoding which can handle such situations.

For this reason, the most probable output sequence $Y$ is the one that has not the single best CTC alignment but the highest sum over the probability of all its possible alignments:

$$P_{\text{CTC}}(Y|X) = \sum_{A \in B^{-1}(Y)} P(A|X) = \sum_{A \in B^{-1}(Y)} \prod_{t=1}^{T} p(a_t|h_t)$$

where $h_t$ is a hidden state of the encoder at timestep $t$.

$$Y = \text{argmax}_Y P_{\text{CTC}}(Y|X)$$

But summing over all alignments is very expensive (there are a lot of alignments).

<img src="images/semi_tree.png" alt="Image 1" style="width: 100%;">


<img src="images/decoding_graph.png" alt="Image 2" style="width: 100%;">

You have only 3 elements in your dictionary! In the real word you would have a huge amount of them. Beam search can help here.

![Image Title](images/beam_search.png)
***Figure 6**: Visualization of the naive beam search.*

If two beams are equal, we simply merge them: we add up the scores and only keep one of the beams.

### Language model

Because of the strong conditional independence assumption mentioned earlier (that the output at time $t$ is independent of the output at time $t − 1$, given the input), CTC does not implicitly learn a language model over the data (unlike the attention-based encoder-decoder architectures). It is, therefore, essential when using CTC to interpolate a language model.

A character-level language model (LM) scores a sequence of characters. We restrict our LM to score single characters (unigram LM) and pairs of characters (bigram LM). We denote a unigram probability of the character $c$ as $P(c)$ and the bigram probability of characters $c_1$, $c_2$ as $P(c_2|c_1)$. The score of a text `hello` is the probability of seeing a single `h`, and the probability of seeing a pair `he`, and a pair `el`, etc.

Training such an LM from a large text is easy: we count the number of times a character occurs and divide by the total number of characters to get the unigram probability. We also count the number of times a pair of characters occurs and normalize it to get the bigram probability.

Just think of a language model as a function taking a sentence as input, which is often only partly constructed, and returning the probability of the last word given all the previous words. What is this good for? Imagine we have the following partly constructed sentence: `Tell us a fairy`. Now we hear the next word but cannot discern whether it should be `tale` or `tail`. These words are homophones, which means they sound alike, although they are spelled differently. A well-trained language model should be able to tell us that `tale` is much more probable than `tail`, making a choice straightforward. You can check out the Wikipedia page on language models for further background reading.

The final formula to receive the score of sequence:

$$score_{\text{CTC}}(Y|X) = \log P_{\text{CTC}}(Y|X) + \lambda_1 \log P_{\text{LM}}(Y) + \lambda_2 L(Y)$$

> **TODO**: What does $L(Y)$ mean?

> **TODO**: Do we need Language Model in Seq2Seq architecture? Why?

### CTC Training

To train a CTC-based ASR system, we use negative log-likelihood loss with a special CTC loss function. Thus the loss for an entire dataset $D$ is the sum of the negative log-likelihoods of the correct output $Y$ for each input $X$:

$$L_{\text{CTC}} = \sum_{(X,Y) \in D} -\log P_{\text{CTC}}(Y|X)$$

#### One more thing before training ASR - Text Normalization. Why do we need it?

Example of Whisper text normalization:

- For English language:
    1. Remove any phrases between matching brackets (`[`, `]`).
    2. Remove any phrases between matching parentheses (`(`, `)`).
    3. Remove any of the following words: `hmm`, `mm`, `mhm`, `mmm`, `uh`, `um`.
    4. Remove whitespace characters that come before an apostrophe `’`.
    5. Convert standard or informal contracted forms of English into the original form (`don't` $\rightarrow$ `do not`).
    6. Remove commas (`,`) between digits.
    7. Remove periods (`.`) not followed by numbers (`It's 9.30 a.m. Are you free...` $\rightarrow$ `It's 9.30 am Are you free...`).
    8. Remove symbols as well as diacritics from the text, where symbols are the characters with the Unicode category starting with M, S, or P, except period, percent, and currency symbols that may be detected in the next step.
    9. Detect any numeric expressions of numbers and currencies and replace them with a form using Arabic numbers, e.g., `Ten thousand dollars` $\rightarrow$ `$10000`.
    10. Convert British spellings into American spellings.
    11. Remove remaining symbols that are not part of any numeric expressions.
    12. Replace any successive whitespace characters with a space.

- For other languages:
    1. Remove any phrases between matching brackets (`[`, `]`).
    2. Remove any phrases between matching parentheses (`(`, `)`).
    3. Replace any markers, symbols, and punctuation characters with a space, i.e., when the Unicode category of each character in the NFKC-normalized string starts with M, S, or P.
    4. Make the text lowercase.
    5. Replace any successive whitespace characters with a space.